# 02 — Preprocessing: Raw → Clean

Transforms the raw JSONL from `data/raw/` into a clean Parquet file in `data/processed/`.

**Steps:**
1. Load & deduplicate
2. Filter noise (empty, too short, too long)
3. Clean text (HTML entities, URLs, whitespace)
4. Detect language — keep English only
5. Save to `data/processed/comments_clean.parquet`
6. Print processing report

In [23]:
import json
import re
import html
from pathlib import Path

import pandas as pd
from langdetect import detect, LangDetectException

ROOT = Path("..")
RAW_DIR   = ROOT / "data" / "raw"
PROC_DIR  = ROOT / "data" / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)

# Load all raw JSONL
records = []
for path in sorted(RAW_DIR.glob("comments_*.jsonl")):
    with open(path, encoding="utf-8") as f:
        for line in f:
            if line.strip():
                records.append(json.loads(line))

df_raw = pd.DataFrame(records)
print(f"Loaded: {len(df_raw):,} raw records from {RAW_DIR}")

Loaded: 143,782 raw records from ..\data\raw


## Step 1 — Deduplicate

In [24]:
df = df_raw.drop_duplicates(subset=["content_id"]).copy()
n_dupes = len(df_raw) - len(df)
print(f"Removed {n_dupes:,} duplicates → {len(df):,} unique comments")

Removed 0 duplicates → 143,782 unique comments


## Step 2 — Text Cleaning

In [25]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # Decode HTML entities (&amp; &lt; &#39; etc.)
    text = html.unescape(text)
    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    # Remove excessive repeated characters (e.g. "looooool" → "lool")
    text = re.sub(r"(.)\1{3,}", r"\1\1", text)
    # Collapse whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["text_clean"] = df["text_raw"].apply(clean_text)
df["word_count"]  = df["text_clean"].str.split().str.len().fillna(0).astype(int)

# Show a few examples where cleaning changed the text
changed = df[df["text_raw"] != df["text_clean"]][["text_raw", "text_clean"]].head(5)
if len(changed):
    print(f"Examples where text was changed ({len(df[df['text_raw'] != df['text_clean']]):,} total):")
    pd.set_option("display.max_colwidth", 100)
    display(changed)
else:
    print("No text needed cleaning (data was already clean)")

Examples where text was changed (29,975 total):


,text_raw,text_clean
0,"US military prepared to strike Iran as early as this weekend, but Trump has yet to make a final ...","US military prepared to strike Iran as early as this weekend, but Trump has yet to make a final ..."
1,"Dump is unhinge, he about to go for broke. The Zionst got him under pressure 😮","Dump is unhinge, he about to go for broke. The Zionst got him under pressure 😮"
2,Have to distract from epstein....,Have to distract from epstein..
3,"Think about consequences, only weak people still believe Trump 's words","Think about consequences, only weak people still believe Trump 's words"
6,Why is US so interested to attack Iran. Why cant US mind it's own business.,Why is US so interested to attack Iran. Why cant US mind it's own business.


## Step 3 — Filter Noise

In [26]:
n_before = len(df)

# Remove empty
df = df[df["text_clean"].str.strip() != ""]
n_empty = n_before - len(df)

# Remove very short (< 3 words) — not enough signal for a classifier
df = df[df["word_count"] >= 3]
n_short = n_before - n_empty - len(df)

# Remove very long (> 300 words) — outliers that break tokenizer max_length
df = df[df["word_count"] <= 300]
n_long = n_before - n_empty - n_short - len(df)

print(f"Removed {n_empty:,} empty")
print(f"Removed {n_short:,} too short (<3 words)")
print(f"Removed {n_long:,} too long (>300 words)")
print(f"Remaining: {len(df):,}")

Removed 71 empty
Removed 20,688 too short (<3 words)
Removed 291 too long (>300 words)
Remaining: 122,732


## Step 4 — Language Detection (keep English only)

In [27]:
def detect_lang(text: str) -> str:
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

print("Detecting language (this takes ~1-2 min for 10k records)...")
df["lang"] = df["text_clean"].apply(detect_lang)

lang_counts = df["lang"].value_counts()
print("\nLanguage distribution (top 10):")
print(lang_counts.head(10).to_string())

n_before_lang = len(df)
df = df[df["lang"] == "en"].copy()
n_non_en = n_before_lang - len(df)
print(f"\nRemoved {n_non_en:,} non-English comments")
print(f"English comments remaining: {len(df):,}")

Detecting language (this takes ~1-2 min for 10k records)...

Language distribution (top 10):
lang
en    107987
es      1523
de      1036
fr       960
af       803
id       719
ru       679
pt       673
it       656
no       556

Removed 14,745 non-English comments
English comments remaining: 107,987


## Step 5 — Finalize Schema & Save

In [28]:
# Keep only the columns needed for training + metadata
KEEP_COLS = [
    "content_id", "platform", "video_id", "parent_id",
    "text_raw", "text_clean", "word_count", "lang",
    "like_count", "reply_count",
    "published_at", "collected_at",
    "channel_id", "channel_name", "channel_category", "video_title",
    "label_toxicity", "label_hate_racism", "label_harassment", "model_version",
]
df = df[[c for c in KEEP_COLS if c in df.columns]]

# Parse timestamps
for col in ["published_at", "collected_at"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], utc=True)

out_path = PROC_DIR / "comments_clean.parquet"
df.to_parquet(out_path, index=False)

print(f"Saved: {out_path}")
print(f"Shape: {df.shape}")
print(f"File size: {out_path.stat().st_size / 1024:.1f} KB")

Saved: ..\data\processed\comments_clean.parquet
Shape: (107987, 20)
File size: 21411.2 KB


## Step 6 — Processing Report

In [29]:
import plotly.express as px

n_raw    = len(df_raw)
n_clean  = len(df)
pct_kept = n_clean / n_raw * 100

print("=" * 50)
print("PREPROCESSING REPORT")
print("=" * 50)
print(f"  Raw records:       {n_raw:,}")
print(f"  After dedup:       {n_raw - n_dupes:,}  (-{n_dupes:,} dupes)")
print(f"  After cleaning:    {n_raw - n_dupes - n_empty - n_short - n_long:,}  (-{n_empty + n_short + n_long:,} noise)")
print(f"  After lang filter: {n_clean:,}  (-{n_non_en:,} non-English)")
print(f"  Retention rate:    {pct_kept:.1f}%")
print()
print("  By channel:")
print(df["channel_name"].value_counts().to_string())
print()
print(f"  Word count — mean: {df['word_count'].mean():.1f}  median: {df['word_count'].median():.0f}  max: {df['word_count'].max()}")
print()
print("  Labels (expect all null — model hasn't run yet):")
for lbl in ["label_toxicity", "label_hate_racism", "label_harassment"]:
    if lbl in df.columns:
        print(f"    {lbl}: {df[lbl].notna().sum()} labeled")
print()
print(f"  Output: {out_path}")
print("=" * 50)

# Word count distribution of clean data
fig = px.histogram(
    df, x="word_count", nbins=60,
    title="Word Count Distribution (Clean Data)",
    labels={"word_count": "Words per comment"},
    color_discrete_sequence=["steelblue"],
)
fig.show()

PREPROCESSING REPORT
  Raw records:       143,782
  After dedup:       143,782  (-0 dupes)
  After cleaning:    122,732  (-21,050 noise)
  After lang filter: 107,987  (-14,745 non-English)
  Retention rate:    75.1%

  By channel:
channel_name
Fox News                   18544
The Hill                   17604
The Late Show (Colbert)    16384
Jimmy Kimmel Live          12625
MrBeast                     9674
PewDiePie                   8561
CNN                         7784
BBC News                    7550
ABC News                    5260
NBC News                    2619
MSNBC                       1083
MrBeast Gaming               289
ESPN                          10

  Word count — mean: 21.3  median: 13  max: 300

  Labels (expect all null — model hasn't run yet):
    label_toxicity: 0 labeled
    label_hate_racism: 0 labeled
    label_harassment: 0 labeled

  Output: ..\data\processed\comments_clean.parquet


## Next Steps

1. **Download Kaggle datasets** — 5 labeled datasets for baseline model training (see `IMPLEMENTATION_ROADMAP.md`)
2. **Notebook 03** — Train baseline RoBERTa model on Kaggle data
3. **Notebook 04** — Run baseline on `comments_clean.parquet` → generate silver labels